In [72]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from scipy import stats
from sklearn.neighbors import KNeighborsRegressor
import math
import datetime
from sklearn import linear_model 
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import requests
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
from config import *

In [2]:
#!pip install spotipy

In [73]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= client_id,
                                                           client_secret= client_secret))

In [74]:
X = pd.read_csv('X.csv')

In [75]:
%%writefile -a functions.py

def isin_hot(df, title, artist):
    
    if ( (df['dataset']== "H") and (df['title'] == title) and (df['artist']==artist) ):
        return True
    else: 
        return False
    
def get_user_song():
    
    title = input("Please, tell me the title of a song you like: " )
    artist = input("Please tell me the artist: ")
    
    return title, artist

In [79]:
%%writefile -a functions.py

def search_song_id_user(df):
    
    song_ids = []
    
    #for number in range(0,len(df)):
        
    for index in list(range(len(df))):
            artist = df.iloc[index,1] 
            title = df.iloc[index,0]
            try:
                results = sp.search(q="track:"+title+" artist:"+artist,limit=1)
                song_id = results['tracks']['items'][0]['id']
                song_ids.append(song_id)
            except:
                print("We cannot guess with this song!".format(song_id))         
                song_ids.append(np.nan)
                
    song_ids = pd.DataFrame(song_ids)
    return song_ids

In [80]:
df ={'title': ['as it was', 'as time goes by'],
       'artist':['harry styles', 'frank sinatra']}
df=pd.DataFrame(df)
print(df)

             title         artist
0        as it was   harry styles
1  as time goes by  frank sinatra


In [81]:
search_song_id_user(df)

,0
0,4LRPiXqCikLlN15c3yImP7
1,1PhQVR6Bxe2iQoIcjflemQ


In [ ]:
def get_audio_features_user(df):
    song_features = []
    
    for index in list(range(len(df))):
        song_id = df.iloc[index,0]
        try:
            my_dict = sp.audio_features(song_id)[0]
            song_features.append(my_dict)
                
        except:
            print("We cannot guess with this song!".format(song_id))
            my_new_dict = {}
            song_features.append(my_new_dict)
        
     
    song_features = pd.DataFrame(song_features)
    return song_features

In [12]:
features_clustering = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms','time_signature']

user_song_id = []

answer = input("Do you want me to recommend you a song? [1->Yes, 2->No] ")
print()

while answer == '1':
   
    title, artist = get_user_song()
    user_dict = {'title': title, 'artist': artist}
    user_preferences = pd.DataFrame([user_dict])
    user_song_id = search_song_id_user(user_preferences)
 
    #while ( math.isnan(user_song_id[0]) == True ):
        #print("Sorry, this song was not found, could you please give me another one: ?")
        #title, artist = get_user_song()
        #user_dict = {'title': title, 'artist': artist}
        #user_preferences = pd.DataFrame(user_dict, index=user_dict.keys())
        #user_song_id = search_song_id_user(user_preferences)
            
    
    #if ( np.isna(user_song_id[0]) == True ):
    #    print("Sorry, this song was not found, could you please give me another one: ?")
    #    title, artist = get_user_song()
    #    user_dict = {'title': title, 'artist': artist}
    #    user_preferences = pd.DataFrame(user_dict, index=user_dict.keys())
    #    user_song_id = search_song_ids(user_preferences, stepsize=1)
    
    user_audio_features = get_audio_features(user_songs_id[0], stepsize=1) #song_features which is a dataframe
    user_preferences = add_audio_features(user_preferences, user_audio__features) #this returns df_final
    df_final = user_preferences[features_clustering]
    scaler=load("scalers/scaler.pickle")
    df_final_scaled = scaler.transform(df_final)
    best_model = load("models/kmeans_17.pickle")
    user_cluster = best_model.predict(df_final_scaled)[0]
    
    if ( isin_hot(df, title, artist) == True):
        recommended_song = X[(X['dataset']== "H") & (X['cluster'] == user_cluster) & (X['song_id'] != user_song_id)].sample()
        print(recomended_song)
        
    else:
        recommended_song = X[(X['dataset']== "NH") & (X['cluster'] == user_cluster) & (X['song_id'] != user_song_id) ].sample()
        print(recomended_song)
        
        answer = input("Do you want me to recommend you another song? [1->Yes, 2->No] ")
        
else:
    print()
    print("Come back soon!")
    


SyntaxError: invalid syntax (2066806523.py, line 48)

In [52]:
features_clustering = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms','time_signature']

user_song_id = []

answer = input("Do you want me to recommend you a song? [1->Yes, 2->No] ")
print()

while answer == '1':
    
    title, artist = get_user_song()
    user_dict = {'title': title, 'artist': artist}
    user_preferences = pd.DataFrame.from_dict([user_dict])
    user_song_id = search_song_id_user(user_preferences)
    user_audio_features = get_audio_features(user_song_id[0]) 
    user_preferences = add_audio_features(user_preferences, user_audio__features) 
    df_final = user_preferences[features_clustering]
    scaler=load("scalers/scaler.pickle")
    df_final_scaled = scaler.transform(df_final)
    best_model = load("models/kmeans_17.pickle")
    user_cluster = best_model.predict(df_final_scaled)[0]
    
    if ( isin_hot(df, title, artist) == True):
        recommended_song = X[(X['dataset']== "H") & (X['cluster'] == user_cluster) & (X['song_id'] != user_song_id)].sample()
        print(recomended_song)
        
    else:
        recommended_song = X[(X['dataset']== "NH") & (X['cluster'] == user_cluster) & (X['song_id'] != user_song_id) ].sample()
        print(recomended_song)
        
        answer = input("Do you want me to recommend you another song? [1->Yes, 2->No] ")
        
#else:
    #print()
    #print("Come back soon!")
 

Do you want me to recommend you a song? [1->Yes, 2->No] 1

Please, tell me the title of a song you like: as it was
Please tell me the artist: harry styles


IndexingError: Too many indexers